In [1]:
import xlrd, xlwt
import requests
import time
from random import randint
from time import sleep
from xlwt import Workbook

In [2]:
excelPath = ("FullDatabase04152020.xlsx")

In [3]:
workbook = xlrd.open_workbook(excelPath) # read workbook
sheet = workbook.sheet_by_index(0)

In [4]:
firstRow = []
for i in range(sheet.ncols):
    firstRow.append(sheet.cell_value(0, i))

In [5]:
Title, Authors, Journal, DOI = [], [], [], []
excelRow = 0;
for i in range(0, sheet.nrows):
    if sheet.cell_type(i, 10) != xlrd.XL_CELL_EMPTY:
        # filter DOI to be usable
        DOIValue = sheet.cell_value(i, 10)
        DOIOnly = DOIValue.replace("https://doi.org/", "")
        DOIOnly = DOIOnly.replace("http://dx.doi.org/", "")
        DOIOnly = DOIOnly.replace("doi:", "")
        DOIOnly = DOIOnly.split(' ', 1)[0]
        if DOIOnly.startswith("DOI") or DOIOnly.startswith("10"):
            DOI.append(DOIOnly)
            Title.append(sheet.cell_value(i, 0))
            Authors.append(sheet.cell_value(i, 1))
            Journal.append(sheet.cell_value(i, 5))    

In [6]:
wb = Workbook() # write workbook 
filteredSheetForWriting = wb.add_sheet("Filtered Sheet")

In [7]:
# write filtered data
for i in range(len(DOI)):
    filteredSheetForWriting.write(i, 0, Title[i])
    filteredSheetForWriting.write(i, 1, Authors[i])
    filteredSheetForWriting.write(i, 2, Journal[i])
    filteredSheetForWriting.write(i, 3, DOI[i])

In [9]:
wb.save("FilteredDatabase.xls")

In [10]:
# Identify Author's affiliation
# https://api.crossref.org/v1/works/
filteredWorkbook = xlrd.open_workbook("FilteredDatabase.xls")
filteredSheet = filteredWorkbook.sheet_by_index(0)
baseURL = "https://api.crossref.org/v1/works/"

In [11]:
URLS = []
for i in range(1, filteredSheet.nrows):
    URLS.append(baseURL + filteredSheet.cell_value(i,3))

In [ ]:
pageInfo =[]
initialTime = time.time()
for i in range(filteredSheet.nrows):
    startTime = time.time()
    sleep(randint(0,5))
    page = requests.get(URLS[i])
    if page.status_code == 200:
        pageJson = page.json()
        pageInfo.append(pageJson)
        print("Finished job at", i, "for", (time.time() - startTime), "seconds, success!")
    else:
        pageInfo.append("Response not 200")
        print("Finished job at", i, "for", (time.time() - startTime), "seconds, error found")
print("job overall time:", (time.time() - initialTime))
    #filteredSheetForWriting.write(i, 4, pageInfo[i])

Finished job at 0 for 4.911361217498779 seconds, success!
Finished job at 1 for 5.838069438934326 seconds, success!
Finished job at 2 for 8.187580823898315 seconds, success!
Finished job at 3 for 3.295785665512085 seconds, success!
Finished job at 4 for 3.3021223545074463 seconds, success!
Finished job at 5 for 3.583261489868164 seconds, success!
Finished job at 6 for 4.279118061065674 seconds, success!
Finished job at 7 for 1.350353479385376 seconds, success!
Finished job at 8 for 4.298878908157349 seconds, success!
Finished job at 9 for 3.3571648597717285 seconds, success!
Finished job at 10 for 4.7737884521484375 seconds, success!
Finished job at 11 for 1.8930296897888184 seconds, success!
Finished job at 12 for 2.5223946571350098 seconds, success!
Finished job at 13 for 5.405152320861816 seconds, success!
Finished job at 14 for 0.23448514938354492 seconds, success!
Finished job at 15 for 3.5455374717712402 seconds, success!
Finished job at 16 for 2.2732038497924805 seconds, success

Finished job at 137 for 1.1888883113861084 seconds, error found
Finished job at 138 for 5.712857723236084 seconds, success!
Finished job at 139 for 5.252086162567139 seconds, success!
Finished job at 140 for 3.376685857772827 seconds, success!
Finished job at 141 for 3.994497299194336 seconds, success!
Finished job at 142 for 8.386033058166504 seconds, success!
Finished job at 143 for 2.3847718238830566 seconds, success!
Finished job at 144 for 5.567825555801392 seconds, success!
Finished job at 145 for 1.2743396759033203 seconds, success!
Finished job at 146 for 0.2284102439880371 seconds, success!
Finished job at 147 for 4.374714612960815 seconds, success!
Finished job at 148 for 4.221872568130493 seconds, success!
Finished job at 149 for 5.388155698776245 seconds, success!
Finished job at 150 for 3.714951992034912 seconds, success!
Finished job at 151 for 3.2455849647521973 seconds, success!
Finished job at 152 for 4.437979459762573 seconds, success!
Finished job at 153 for 3.811909

Finished job at 272 for 2.6839888095855713 seconds, success!
Finished job at 273 for 10.120906591415405 seconds, success!
Finished job at 274 for 3.2182109355926514 seconds, error found
Finished job at 275 for 5.247203350067139 seconds, success!
Finished job at 276 for 5.30396580696106 seconds, success!
Finished job at 277 for 5.354405164718628 seconds, success!
Finished job at 278 for 5.733402252197266 seconds, success!
Finished job at 279 for 3.419003486633301 seconds, success!
Finished job at 280 for 4.199358224868774 seconds, success!
Finished job at 281 for 5.332840919494629 seconds, success!
Finished job at 282 for 5.167070150375366 seconds, success!
Finished job at 283 for 19.33227229118347 seconds, success!
Finished job at 284 for 1.2897264957427979 seconds, success!
Finished job at 285 for 5.421842575073242 seconds, success!
Finished job at 286 for 3.2791876792907715 seconds, success!
Finished job at 287 for 1.6986279487609863 seconds, success!
Finished job at 288 for 6.685273

Finished job at 407 for 1.9706776142120361 seconds, success!
Finished job at 408 for 4.4039623737335205 seconds, success!
Finished job at 409 for 4.553831338882446 seconds, success!
Finished job at 410 for 1.372450590133667 seconds, success!
Finished job at 411 for 2.267042636871338 seconds, success!
Finished job at 412 for 0.28539037704467773 seconds, success!
Finished job at 413 for 4.304474830627441 seconds, success!
Finished job at 414 for 0.25675129890441895 seconds, success!
Finished job at 415 for 5.907688617706299 seconds, success!
Finished job at 416 for 3.491163969039917 seconds, success!
Finished job at 417 for 2.0226731300354004 seconds, success!
Finished job at 418 for 1.200753927230835 seconds, error found
Finished job at 419 for 0.6827404499053955 seconds, success!
Finished job at 420 for 5.3921427726745605 seconds, success!
Finished job at 421 for 3.4737179279327393 seconds, success!
Finished job at 422 for 5.345042705535889 seconds, success!
Finished job at 423 for 2.5

Finished job at 542 for 4.724185228347778 seconds, success!
Finished job at 543 for 1.5044827461242676 seconds, success!
Finished job at 544 for 4.215457201004028 seconds, success!
Finished job at 545 for 3.3378586769104004 seconds, success!
Finished job at 546 for 1.230090618133545 seconds, error found
Finished job at 547 for 3.2758076190948486 seconds, error found
Finished job at 548 for 0.30298805236816406 seconds, error found
Finished job at 549 for 1.2726585865020752 seconds, success!
Finished job at 550 for 5.3144049644470215 seconds, success!
Finished job at 551 for 0.24222493171691895 seconds, error found
Finished job at 552 for 0.3451969623565674 seconds, success!
Finished job at 553 for 0.3023715019226074 seconds, success!
Finished job at 554 for 4.253323793411255 seconds, error found
Finished job at 555 for 3.276108741760254 seconds, success!
Finished job at 556 for 6.376914739608765 seconds, success!
Finished job at 557 for 1.3317952156066895 seconds, success!
Finished job 

...

Finished job at 2949 for 1.3483514785766602 seconds, success!
Finished job at 2950 for 3.574254035949707 seconds, success!
Finished job at 2951 for 4.393989562988281 seconds, success!
Finished job at 2952 for 0.352200984954834 seconds, success!
Finished job at 2953 for 0.4070301055908203 seconds, success!
Finished job at 2954 for 2.355109691619873 seconds, success!
Finished job at 2955 for 2.2553963661193848 seconds, success!
Finished job at 2956 for 6.333165168762207 seconds, success!
Finished job at 2957 for 8.100393056869507 seconds, success!
Finished job at 2958 for 5.531805753707886 seconds, success!
Finished job at 2959 for 3.454726457595825 seconds, success!
Finished job at 2960 for 4.909019231796265 seconds, success!
Finished job at 2961 for 3.6205499172210693 seconds, success!
Finished job at 2962 for 8.28446626663208 seconds, success!
Finished job at 2963 for 3.177945613861084 seconds, success!
Finished job at 2964 for 1.9273509979248047 seconds, success!
Finished job at 2965

Finished job at 3083 for 6.116104602813721 seconds, success!
Finished job at 3084 for 3.285080909729004 seconds, success!
Finished job at 3085 for 5.220126390457153 seconds, error found
Finished job at 3086 for 5.230293035507202 seconds, error found
Finished job at 3087 for 1.2276616096496582 seconds, error found
Finished job at 3088 for 2.265070676803589 seconds, success!
Finished job at 3089 for 1.2668085098266602 seconds, error found
Finished job at 3090 for 0.204085111618042 seconds, error found
Finished job at 3091 for 4.289837121963501 seconds, success!
Finished job at 3092 for 5.387577533721924 seconds, success!
Finished job at 3093 for 4.105057001113892 seconds, success!
Finished job at 3094 for 2.2425222396850586 seconds, success!
Finished job at 3095 for 1.4086787700653076 seconds, success!
Finished job at 3096 for 3.5492007732391357 seconds, success!
Finished job at 3097 for 7.179389476776123 seconds, success!
Finished job at 3098 for 8.358112812042236 seconds, success!
Fini

Finished job at 3216 for 4.341277599334717 seconds, success!
Finished job at 3217 for 2.34061598777771 seconds, success!
Finished job at 3218 for 2.026366949081421 seconds, success!
Finished job at 3219 for 7.235931873321533 seconds, success!
Finished job at 3220 for 4.674502372741699 seconds, success!
Finished job at 3221 for 6.407909393310547 seconds, success!
Finished job at 3222 for 2.322075843811035 seconds, success!
Finished job at 3223 for 2.5459163188934326 seconds, success!
Finished job at 3224 for 5.999628305435181 seconds, success!
Finished job at 3225 for 7.039719820022583 seconds, success!
Finished job at 3226 for 3.339430809020996 seconds, success!
Finished job at 3227 for 0.24500203132629395 seconds, success!
Finished job at 3228 for 0.23691630363464355 seconds, success!
Finished job at 3229 for 3.2695090770721436 seconds, success!
Finished job at 3230 for 3.3172433376312256 seconds, success!
Finished job at 3231 for 5.175166845321655 seconds, success!
Finished job at 32

Finished job at 3350 for 9.29250192642212 seconds, success!
Finished job at 3351 for 2.82550311088562 seconds, success!
Finished job at 3352 for 2.362779140472412 seconds, success!
Finished job at 3353 for 0.3635213375091553 seconds, success!
Finished job at 3354 for 1.2110559940338135 seconds, error found
Finished job at 3355 for 4.286471605300903 seconds, success!
Finished job at 3356 for 2.748159646987915 seconds, success!
Finished job at 3357 for 1.635190486907959 seconds, success!
Finished job at 3358 for 2.325741767883301 seconds, success!
Finished job at 3359 for 4.271153688430786 seconds, success!
Finished job at 3360 for 3.382977247238159 seconds, success!
Finished job at 3361 for 3.25485897064209 seconds, success!
Finished job at 3362 for 3.8794028759002686 seconds, success!
Finished job at 3363 for 5.790595054626465 seconds, success!
Finished job at 3364 for 4.2267937660217285 seconds, error found
Finished job at 3365 for 2.1952409744262695 seconds, error found
Finished job 

Finished job at 3483 for 4.344112396240234 seconds, success!
Finished job at 3484 for 4.259307384490967 seconds, success!
Finished job at 3485 for 4.24878191947937 seconds, success!
Finished job at 3486 for 4.282048940658569 seconds, success!
Finished job at 3487 for 1.9243354797363281 seconds, success!
Finished job at 3488 for 8.74960207939148 seconds, success!
Finished job at 3489 for 3.446942090988159 seconds, success!
Finished job at 3490 for 8.987887859344482 seconds, success!
Finished job at 3491 for 6.705240964889526 seconds, success!
Finished job at 3492 for 2.0571980476379395 seconds, success!
Finished job at 3493 for 2.479327917098999 seconds, success!
Finished job at 3494 for 5.36226487159729 seconds, success!
Finished job at 3495 for 3.3055529594421387 seconds, success!
Finished job at 3496 for 1.2883436679840088 seconds, success!
Finished job at 3497 for 2.261918544769287 seconds, success!
Finished job at 3498 for 5.375852346420288 seconds, success!
Finished job at 3499 fo

In [77]:
# indexing off by two for excel
for i in range(len(pageInfo)):
    if pageInfo[i] == "Response not 200":
        print("Error at", i+2)
    else:
        filteredSheetForWriting.write(i, 5, str(pageInfo[i].get("message").get("author")[0].get("affiliation")))
        print(pageInfo[i].get("message").get("author")[0].get("affiliation"), "index", i+2)

[{'name': 'Department of Clinical Medicine, School of MedicineZhejiang University City College Hangzhou China'}] index 2
[{'name': 'Department of Medicine, University of Toronto, Toronto, Canada'}, {'name': 'Divisions of General Internal Medicine and Infectious Diseases, University Health Network, Toronto, Canada'}] index 3
[{'name': 'From the NHC Key Laboratory of Biosafety, National Institute for Viral Disease Control and Prevention, Chinese Center for Disease Control and Prevention (N.Z., W.W., J.S., X.Z., B.H., R.L., P.N., X.M., D.W., W.X., G.W., G.F.G., W.T.), and the Department of Infectious Diseases, Beijing Ditan Hospital, Capital Medical University (X.L.) — both in Beijing; Wuhan Jinyintan Hospital (D.Z.), the Division for Viral Disease Detection, Hubei Provincial Center for Disease Control and Prevention (B.Y., F.Z.), and...'}] index 4
[{'name': 'From the Laboratory of Virology, National Institute of Allergy and Infectious Diseases, National Institutes of Health, Hamilton, MT

In [78]:
wb.save("FilteredDatabase.xls")